# Trial 1 - Execution failure

In [1]:
! pip install transformers datasets

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
data = [
    {"input": "Find all users older than 30", "output": "db.users.find({ age: { $gt: 30 } })"},
    {"input": "Find users from New York", "output": "db.users.find({ 'address.city': 'New York' })"},
]

In [3]:
# Tokenize Data
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")  # Example tokenizer

# Tokenize the data
inputs = [example["input"] for example in data]
outputs = [example["output"] for example in data]

# Tokenize inputs and outputs
tokenized_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")
tokenized_outputs = tokenizer(outputs, padding=True, truncation=True, return_tensors="pt")

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--codellama--CodeLlama-7b-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an adm

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
# Create Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments

# Load model and tokenizer
model_name = "codellama/CodeLlama-7b-hf"  # Example model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Prepare dataset
train_dataset = ...  # Your preprocessed dataset

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
)

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [ ]:
# Fine-tune the model
trainer.train()

# Trial 2 - Incorrect response from model - training error - need analysis

## Library Installations

In [4]:
! pip install pymongo transformers torch datasets


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# To Fix Model specific errors
! pip install accelerate>=0.26.0


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
! pip install transformers[torch]


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
! pip show accelerate

Name: accelerate
Version: 1.3.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: c:\users\user\appdata\local\programs\python\python39\lib\site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [25]:
! pip install fastapi


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Creating datasets

In [27]:
data = [
    {
        "question": "Find all users older than 30.",
        "query": "db.users.find({ age: { $gt: 30 } })"
    },
    {
        "question": "Get the names of users with the email domain 'example.com'.",
        "query": "db.users.find({ email: { $regex: 'example.com$' } }, { name: 1, _id: 0 })"
    },
    {
        "question": "Count the number of users.",
        "query": "db.users.countDocuments()"
    }
]

In [41]:
data = [
    {"question": "Find all users older than 30.", "query": "db.users.find({ age: { $gt: 30 } })"},
    {"question": "Get the names of users with the email domain 'example.com'.", "query": "db.users.find({ email: { $regex: 'example.com$' } }, { name: 1, _id: 0 })"},
    {"question": "Count the number of users.", "query": "db.users.countDocuments()"}
]

In [42]:
import json
with open("question_query_pairs.json", "w") as f:
    json.dump(data, f)

## Preprocess Dataset

In [43]:
# Load the dataset
from datasets import load_dataset

dataset = load_dataset("json", data_files="question_query_pairs.json")

Generating train split: 3 examples [00:00, 136.37 examples/s]


In [44]:
# Split the dataset into train and validation sets
split_dataset = dataset["train"].train_test_split(test_size=0.2)  # 20% for validation

In [45]:
split_dataset["validation"] = split_dataset.pop("test")  # Rename "test" to "validation"

In [46]:
# # Tokenize the Data
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("gpt2")

# def preprocess_function(examples):
#     inputs = [q for q in examples["question"]]
#     targets = [q for q in examples["query"]]
#     model_inputs = tokenizer(inputs, max_length=128, truncation=True)
#     labels = tokenizer(targets, max_length=128, truncation=True)
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# tokenized_dataset = dataset.map(preprocess_function, batched=True)

####################################################
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("t5-small")

# def preprocess_function(examples):
#     inputs = [q for q in examples["question"]]
#     targets = [q for q in examples["query"]]
#     model_inputs = tokenizer(inputs, max_length=128, truncation=True)
#     labels = tokenizer(targets, max_length=128, truncation=True)
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# tokenized_dataset = split_dataset.map(preprocess_function, batched=True)

#####################################################

def preprocess_function(examples):
    # Tokenize inputs (questions)
    inputs = tokenizer(
        examples["question"],
        max_length=128,  # Set a maximum length
        truncation=True,  # Truncate sequences longer than max_length
        padding="max_length",  # Pad sequences to max_length
    )

    # Tokenize labels (queries)
    labels = tokenizer(
        examples["query"],
        max_length=128,  # Set a maximum length
        truncation=True,  # Truncate sequences longer than max_length
        padding="max_length",  # Pad sequences to max_length
    )

    # Add the tokenized labels to the inputs
    inputs["labels"] = labels["input_ids"]
    return inputs


In [47]:
tokenized_dataset = split_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map: 100%|██████████| 1/1 [00:00<00:00,  4.76 examples/s]


In [48]:
print(tokenized_dataset["train"][0])  # Inspect the first example

{'question': "Get the names of users with the email domain 'example.com'.", 'query': "db.users.find({ email: { $regex: 'example.com$' } }, { name: 1, _id: 0 })", 'input_ids': [1609, 8, 3056, 13, 1105, 28, 8, 791, 3303, 3, 31, 994, 9, 9208, 5, 287, 31, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 

## Fine-Tune a Pre-Trained Model

In [49]:
# Load a Pre-Trained Model:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [50]:
# Set Up Training Arguments:
# from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# training_args = Seq2SeqTrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     save_total_limit=2,
#     predict_with_generate=True,
# )

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
)

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [51]:
# Define the Trainer:
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["validation"],
#     tokenizer=tokenizer,
# )

#######################################

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["validation"],  # Use the validation split
#     tokenizer=tokenizer,
# )

########################################

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

C:\Users\user\AppData\Local\Temp\ipykernel_23424\1467447618.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [52]:
trainer.train()

 33%|███▎      | 1/3 [00:08<00:16,  8.18s/it]

{'eval_loss': 18.360090255737305, 'eval_runtime': 0.516, 'eval_samples_per_second': 1.938, 'eval_steps_per_second': 1.938, 'epoch': 1.0}


 67%|██████▋   | 2/3 [00:15<00:07,  7.23s/it]

{'eval_loss': 18.206106185913086, 'eval_runtime': 0.797, 'eval_samples_per_second': 1.255, 'eval_steps_per_second': 1.255, 'epoch': 2.0}


100%|██████████| 3/3 [00:28<00:00,  9.59s/it]

{'eval_loss': 18.127771377563477, 'eval_runtime': 0.68, 'eval_samples_per_second': 1.471, 'eval_steps_per_second': 1.471, 'epoch': 3.0}
{'train_runtime': 28.7953, 'train_samples_per_second': 0.208, 'train_steps_per_second': 0.104, 'train_loss': 11.472581227620443, 'epoch': 3.0}


TrainOutput(global_step=3, training_loss=11.472581227620443, metrics={'train_runtime': 28.7953, 'train_samples_per_second': 0.208, 'train_steps_per_second': 0.104, 'total_flos': 203012702208.0, 'train_loss': 11.472581227620443, 'epoch': 3.0})

## Evaluate model

In [53]:
# # Generate Queries from Questions

# def generate_query(question):
#     inputs = tokenizer(question, return_tensors="pt", max_length=128, truncation=True)
#     outputs = model.generate(inputs["input_ids"], max_length=128)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# question = "Find all users older than 30."
# generated_query = generate_query(question)
# print(generated_query)  # Output: db.users.find({ age: { $gt: 30 } })

def generate_query(question):
    # Tokenize the input question
    inputs = tokenizer(question, return_tensors="pt", max_length=128, truncation=True)
    print("Tokenized Input:", inputs)

    # Generate the output
    outputs = model.generate(inputs["input_ids"], max_length=128)
    print("Generated Output IDs:", outputs)

    # Decode the output
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Decoded Output:", decoded_output)

    return decoded_output

question = "Find all users older than 30."
generated_query = generate_query(question)
print("Final Output:", generated_query)

Tokenized Input: {'input_ids': tensor([[2588,   66, 1105, 2749,  145,  604,    5,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
Generated Output IDs: tensor([[   0, 2588,   66, 1105, 2749,  145,  604,    5,    1]])
Decoded Output: Find all users older than 30.
Final Output: Find all users older than 30.


In [23]:
# Evaluate Execution Accuracy
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["your_database"]
collection = db["users"]

result = collection.find({ "age": { "$gt": 30 } })
for doc in result:
    print(doc)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 679d094fd4af20255ebbc67e, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

## Deploy Model

In [21]:
# Save the model
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

('./trained_model\\tokenizer_config.json',
 './trained_model\\special_tokens_map.json',
 './trained_model\\spiece.model',
 './trained_model\\added_tokens.json',
 './trained_model\\tokenizer.json')

In [26]:
# Creating API
# Ruuning the API - uvicorn api:app --reload
from fastapi import FastAPI
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

app = FastAPI()

model = AutoModelForSeq2SeqLM.from_pretrained("./trained_model")
tokenizer = AutoTokenizer.from_pretrained("./trained_model")

@app.post("/generate_query")
async def generate_query(question: str):
    inputs = tokenizer(question, return_tensors="pt", max_length=128, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=128)
    query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"query": query}

# Trial 3 - increasing training set - did not help

In [54]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset

# Load the dataset
data = [
    {"question": "Find all users older than 30.", "query": "db.users.find({ age: { $gt: 30 } })"},
    {"question": "Get the names of users with the email domain 'example.com'.", "query": "db.users.find({ email: { $regex: 'example.com$' } }, { name: 1, _id: 0 })"},
    {"question": "Count the number of users.", "query": "db.users.countDocuments()"}
]
dataset = Dataset.from_dict({"question": [d["question"] for d in data], "query": [d["query"] for d in data]})

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Tokenize the dataset
def preprocess_function(examples):
    inputs = tokenizer(examples["question"], max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(examples["query"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
)

# Define the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map: 100%|██████████| 3/3 [00:00<00:00, 34.48 examples/s]
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_23424\522395396.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq

{'eval_loss': 14.79012680053711, 'eval_runtime': 1.473, 'eval_samples_per_second': 2.037, 'eval_steps_per_second': 0.679, 'epoch': 1.0}


                                             
 67%|██████▋   | 2/3 [00:35<00:15, 15.44s/it]

{'eval_loss': 14.529919624328613, 'eval_runtime': 2.6284, 'eval_samples_per_second': 1.141, 'eval_steps_per_second': 0.38, 'epoch': 2.0}


                                             
100%|██████████| 3/3 [00:55<00:00, 14.31s/it]

{'eval_loss': 14.394962310791016, 'eval_runtime': 2.2388, 'eval_samples_per_second': 1.34, 'eval_steps_per_second': 0.447, 'epoch': 3.0}
{'train_runtime': 55.5331, 'train_samples_per_second': 0.162, 'train_steps_per_second': 0.054, 'train_loss': 12.3587646484375, 'epoch': 3.0}


100%|██████████| 3/3 [00:55<00:00, 18.47s/it]


('./trained_model\\tokenizer_config.json',
 './trained_model\\special_tokens_map.json',
 './trained_model\\spiece.model',
 './trained_model\\added_tokens.json')

In [55]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./trained_model")
tokenizer = T5Tokenizer.from_pretrained("./trained_model")

# Generate queries
def generate_query(question):
    inputs = tokenizer(question, return_tensors="pt", max_length=128, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

question = "Find all users older than 30."
generated_query = generate_query(question)
print(generated_query)  # Expected Output: db.users.find({ age: { $gt: 30 } })

Find all users older than 30.


# Trial 4 - Tried with Facebook bart model - not working

In [56]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--facebook--bart-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [58]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset

# Load the dataset
data = [
    {"question": "Find all users older than 30.", "query": "db.users.find({ age: { $gt: 30 } })"},
    {"question": "Get the names of users with the email domain 'example.com'.", "query": "db.users.find({ email: { $regex: 'example.com$' } }, { name: 1, _id: 0 })"},
    {"question": "Count the number of users.", "query": "db.users.countDocuments()"},
    # Add more examples
]
dataset = Dataset.from_dict({"question": [d["question"] for d in data], "query": [d["query"] for d in data]})

# Tokenize the dataset
def preprocess_function(examples):
    inputs = tokenizer(
        examples["question"],
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    labels = tokenizer(
        examples["query"],
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,  # Increase the number of epochs
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
)

# Define the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

Map: 100%|██████████| 3/3 [00:00<00:00, 17.86 examples/s]
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_23424\1918723563.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
                                              
 10%|█         | 1/10 [00:20<02:37, 17.48s/it]

{'eval_loss': 10.97472095489502, 'eval_runtime': 3.1891, 'eval_samples_per_second': 0.941, 'eval_steps_per_second': 0.314, 'epoch': 1.0}


                                              
 20%|██        | 2/10 [00:39<02:24, 18.04s/it]

{'eval_loss': 10.726054191589355, 'eval_runtime': 3.1684, 'eval_samples_per_second': 0.947, 'eval_steps_per_second': 0.316, 'epoch': 2.0}


                                              
 30%|███       | 3/10 [00:56<02:04, 17.84s/it]

...

                                              
 90%|█████████ | 9/10 [02:28<00:14, 14.77s/it]

{'eval_loss': 9.609941482543945, 'eval_runtime': 2.2448, 'eval_samples_per_second': 1.336, 'eval_steps_per_second': 0.445, 'epoch': 9.0}


                                               
100%|██████████| 10/10 [02:49<00:00, 16.97s/it]


{'eval_loss': 9.57713794708252, 'eval_runtime': 1.694, 'eval_samples_per_second': 1.771, 'eval_steps_per_second': 0.59, 'epoch': 10.0}
{'train_runtime': 169.8347, 'train_samples_per_second': 0.177, 'train_steps_per_second': 0.059, 'train_loss': 10.626959228515625, 'epoch': 10.0}


('./trained_model\\tokenizer_config.json',
 './trained_model\\special_tokens_map.json',
 './trained_model\\vocab.json',
 './trained_model\\merges.txt',
 './trained_model\\added_tokens.json')

In [59]:
def generate_query(question):
    inputs = tokenizer(question, return_tensors="pt", max_length=128, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

question = "Find all users older than 30."
generated_query = generate_query(question)
print(generated_query)  # Expected Output: db.users.find({ age: { $gt: 30 } })

Find all users older than 30.
